# Planck Quantum SDK - Quick Start

This notebook demonstrates how to use the Planck SDK to run quantum circuits remotely.

**Requirements:**
- A Planck API key (get one at https://www.plancktechnologies.xyz/qsaas/settings)

---

## 1. Install the SDK

Install from PyPI - the simplest method!

In [ ]:
# Install Planck SDK from PyPI
!pip install -q planck_sdk

# Verify installation
import planck_sdk
print(f"Planck SDK v{planck_sdk.__version__} installed successfully!")

## 2. Import and Configure

Replace `YOUR_API_KEY` with your actual Planck API key.

In [ ]:
from planck_sdk import PlanckClient, QuantumCircuit, ExecutionResult

# Option 1: Direct API key
API_KEY = "YOUR_API_KEY"  # Replace with your API key

# Option 2: Use Colab secrets (recommended for security)
# from google.colab import userdata
# API_KEY = userdata.get('PLANCK_API_KEY')

# Initialize the client
client = PlanckClient(
    api_key=API_KEY,
    base_url="https://www.plancktechnologies.xyz"
)

# Test connection
if client.ping():
    print("Connected to Planck API successfully!")
else:
    print("Connection failed. Check your API key.")

## 3. Run Your First Quantum Circuit

Let's run a simple VQE (Variational Quantum Eigensolver) algorithm with sample data.

In [ ]:
# Sample data for quantum processing
my_data = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

# Run the quantum circuit
result = client.run(
    data=my_data,
    algorithm="vqe",
    shots=1024
)

print(f"Execution completed!")
print(f"Execution ID: {result.execution_id}")
print(f"Runtime: {result.runtime_ms:.1f}ms")
print(f"Fidelity: {result.fidelity:.3f}")
print(f"\nTop measurement results:")
for state, count in sorted(result.counts.items(), key=lambda x: -x[1])[:5]:
    prob = count / result.shots * 100
    print(f"  |{state}>: {count} ({prob:.1f}%)")

In [ ]:
# Visualize results with histogram
result.plot_histogram(top_n=8)

## 4. Try Different Algorithms

Available algorithms:
- `vqe` - Variational Quantum Eigensolver (optimization)
- `grover` - Grover's Search (database search)
- `qaoa` - QAOA (combinatorial optimization)
- `qft` - Quantum Fourier Transform
- `bell` - Bell State (entanglement)
- `shor` - Shor's Algorithm (factorization)

In [ ]:
# Bell State - Quantum Entanglement
bell_result = client.run(
    data=[1, 0],
    algorithm="bell",
    shots=2048
)

print("Bell State Results (expect |00> and |11> to be equally probable):")
bell_result.plot_histogram()

In [ ]:
# Grover's Search Algorithm
grover_result = client.run(
    data=[0, 1, 0, 1, 1, 0, 1, 0],  # Search pattern
    algorithm="grover",
    shots=2048
)

print("Grover's Search Results:")
print(f"Fidelity: {grover_result.fidelity:.3f}")
grover_result.plot_histogram(top_n=5)

In [ ]:
# QAOA for optimization
qaoa_result = client.run(
    data=[1.5, 2.3, 0.8, 3.1, 1.9],
    algorithm="qaoa",
    shots=1024,
    error_mitigation="high"
)

print("QAOA Results:")
print(f"Fidelity: {qaoa_result.fidelity:.3f}")
print(f"Runtime: {qaoa_result.runtime_ms:.1f}ms")
qaoa_result.plot_histogram(top_n=5)

## 5. Generate Circuit Without Executing

You can generate the QASM code without running it.

In [ ]:
# Generate circuit only
circuit = client.generate_circuit(
    data=[1, 2, 3, 4],
    algorithm="grover",
    qubits=4
)

print(f"Generated Circuit:")
print(f"  Qubits: {circuit.qubits}")
print(f"  Depth: {circuit.depth}")
print(f"  Gates: {circuit.gate_count}")
print(f"  Recommended shots: {circuit.recommended_shots}")
print(f"\nQASM Code (first 15 lines):")
print("\n".join(circuit.qasm.split("\n")[:15]))

## 6. Transpile and Visualize Circuits

In [ ]:
# Transpile for quantum hardware
transpiled = client.transpile(
    qasm=circuit.qasm,
    backend="quantum_qpu",
    qubits=circuit.qubits
)

print(f"Transpilation Results:")
print(f"  SWAP gates added: {transpiled['swap_count']}")
print(f"  New depth: {transpiled['depth']}")
print(f"  Qubit mapping: {transpiled['mapped_qubits']}")

In [ ]:
# Visualize circuit
viz = client.visualize(qasm=circuit.qasm)

print(f"Circuit Visualization:")
print(f"  Format: {viz['format']}")
print(f"  Size: {viz['width']}x{viz['height']}px")
print(f"  Stats: {viz['stats']}")

# Display SVG in notebook
from IPython.display import SVG, display
display(SVG(viz['image_data']))

## 7. Get ML-Powered Recommendations

The platform uses machine learning to recommend optimal execution parameters.

In [ ]:
# Get ML recommendations
recommendations = client.get_recommendations(
    qubits=8,
    depth=15,
    gate_count=40,
    algorithm="vqe",
    data_size=100
)

print("ML Recommendations:")
print(f"  Recommended shots: {recommendations['recommended_shots']}")
print(f"  Recommended backend: {recommendations['recommended_backend']}")
print(f"  Error mitigation: {recommendations['recommended_error_mitigation']}")
print(f"  Confidence: {recommendations['confidence']:.1%}")
print(f"  Reasoning: {recommendations.get('reasoning', 'N/A')}")

## 8. Get Digital Twin Insights

AI-powered analysis of your quantum circuit execution.

In [ ]:
# Get digital twin insights
insights = client.get_digital_twin(
    algorithm="vqe",
    circuit_info={
        "qubits": result.circuit.qubits if result.circuit else 4,
        "gates": result.circuit.gate_count if result.circuit else 10,
        "depth": result.circuit.depth if result.circuit else 5
    },
    execution_results={
        "probabilities": result.probabilities,
        "counts": result.counts,
        "execution_id": result.execution_id
    },
    backend_config={
        "shots": result.shots,
        "errorMitigation": "medium"
    }
)

print("Digital Twin Analysis:")
print(f"\nInterpretation: {insights.get('algorithm_interpretation', 'N/A')}")
print(f"\nKey Findings:")
for finding in insights.get('key_findings', []):
    print(f"  - {finding}")
print(f"\nStatistical Analysis:")
stats = insights.get('statistical_analysis', {})
print(f"  Entropy: {stats.get('entropy', 'N/A')}")
print(f"  Dominant state: {stats.get('dominant_state', 'N/A')}")
print(f"  Max probability: {stats.get('max_probability', 'N/A')}%")
print(f"\nRecommendations:")
for rec in insights.get('recommendations', []):
    print(f"  - {rec}")

## 9. Export and Save Results

Save your results for later analysis.

In [ ]:
# Get detailed result info
print(f"Execution Summary:")
print(f"  Execution ID: {result.execution_id}")
print(f"  Algorithm: {result.algorithm}")
print(f"  Backend: {result.backend}")
print(f"  Shots: {result.shots}")
print(f"  Fidelity: {result.fidelity:.3f}")
print(f"  Runtime: {result.runtime_ms:.1f}ms")
print(f"  Unique states: {len(result.counts)}")

# Export to JSON
result_json = result.to_json()
print(f"\nJSON export ready ({len(result_json)} bytes)")

# Save to file
result.save("my_quantum_result.json")
print("Saved to my_quantum_result.json")

# Save circuit QASM
if result.circuit:
    result.circuit.save("my_circuit.qasm")
    print("Circuit saved to my_circuit.qasm")

---

## Next Steps

- **Get your API key**: https://www.plancktechnologies.xyz/qsaas/settings
- **Full documentation**: https://github.com/HectorNaaa/Planck-QSaaS/tree/main/sdk/python
- **Platform dashboard**: https://www.plancktechnologies.xyz/qsaas/dashboard
- **API reference**: https://github.com/HectorNaaa/Planck-QSaaS/blob/main/API.md

Happy quantum computing!